In [1]:
## Trained model for story generator - using the gpt_2_simple
## Forked the original github repo to my repo - https://github.com/rbewoor/gpt-2-simple

## Anaconda environment setup as follows:
#	*** create new environment
#conda create -n --newENV python=3.7
#
#	*** general stuff
#conda install jupyter
#
#	*** Story Generation
#conda install numpy
#conda install requests
#conda install regex
#pip install tensorflow==1.15.2
#** not done this so far pip install toposort
#pip install gpt-2-simple			## Successfully installed gpt-2-simple-0.7.1 regex-2020.10.23 toposort-1.5

In [1]:
import os
import json

import gpt_2_simple as gpt2
import tensorflow as tf

from gpt_2_simple.src import model, sample, encoder, memory_saving_gradients
from gpt_2_simple.src.load_dataset import load_dataset, Sampler
from gpt_2_simple.src.accumulate import AccumulatingOptimizer

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
assert tf.__version__ < '2.0.0', "gpt-2-simple currently does not support " \
    "TensorFlow 2.0. You'll need to use a virtualenv/cloud computer which " \
    "has Tensorflow 1.X on it."

In [7]:
###############################################################################################################
## -------------------     STORY GENERATOR STARTS                  STORY GENERATOR STARTS   -------------------
###############################################################################################################
def my_load_gpt2(_sess,
              _checkpoint='latest',
              _run_name="run1",
              _checkpoint_dir="checkpoint",
              _model_name=None,
              _model_dir='models',
              _multi_gpu=False):
    """Loads the model checkpoint or existing model into a TensorFlow session for repeated predictions.
    """

    if _model_name:
        checkpoint_path = os.path.join(_model_dir, _model_name)
    else:
        checkpoint_path = os.path.join(_checkpoint_dir, _run_name)
    print(f"\ncheckpoint_path = {checkpoint_path}\n")

    hparams = model.default_hparams()
    with open(os.path.join(checkpoint_path, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    context = tf.compat.v1.placeholder(tf.int32, [1, None])

    gpus = []
    if _multi_gpu:
        gpus = get_available_gpus()

    output = model.model(hparams=hparams, X=context, gpus=gpus)

    if _checkpoint=='latest':
        ckpt = tf.train.latest_checkpoint(checkpoint_path)
    else:
        ckpt = os.path.join(checkpoint_path,_checkpoint)

    saver = tf.compat.v1.train.Saver(allow_empty=True)
    _sess.run(tf.compat.v1.global_variables_initializer())

    if _model_name:
        print(f"\nLoading pretrained model :: {ckpt}\n")
    else:
        print(f"\nLoading checkpoint :: {ckpt}\n")
    saver.restore(_sess, ckpt)

def reset_session(_sess, threads=-1, server=None):
    """Resets the current TensorFlow session, to clear memory
    or load another model.
    """

    tf.compat.v1.reset_default_graph()
    _sess.close()

def sty_gen_inference_functionality(_sg_params, _DEBUG_SWITCH):
    _seed_for_styGen, SG_LENGTH, SG_TEMPERATURE, SG_NSAMPLES, SG_BATCH_SIZE, RUN_NAME, RETURN_LIST, DEST_PATH = _sg_params
    
    sty_gen_inference_module_results = []
    
    
    ## set up the pre-trained GPT-2 model from checkpoint directory
    
    ## copy checkpoint directory from gdrive - not required as saved it locally
    ##gpt2.copy_checkpoint_from_gdrive(run_name='run1')
    GPT2_CHECKPOINT_DIR = r'/home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/'
    #RUN_NAME = r"run1"

    sess = gpt2.start_tf_sess()
    print(f"\nTF session started\n")
    #gpt2.load_gpt2(sess, run_name='run1')
    my_load_gpt2(
        _sess = sess,
        _checkpoint='latest',
        _run_name=RUN_NAME,
        _checkpoint_dir=GPT2_CHECKPOINT_DIR,
        _model_name=None,
        _model_dir='models',
        _multi_gpu=False)
    
    print(f"\n\nGenerating the story using 'prefix' = {_seed_for_styGen}\n")
    sty_gen_inference_module_results = gpt2.generate(
        sess,
        length=SG_LENGTH,
        temperature=SG_TEMPERATURE,
        prefix=seed_for_styGen,
        nsamples=SG_NSAMPLES,
        batch_size=SG_BATCH_SIZE,
        checkpoint_dir=GPT2_CHECKPOINT_DIR,
        return_as_list=RETURN_LIST,
        destination_path=DEST_PATH
    )
    
    reset_session(sess)
    print(f"\n\nFINISHED generating the story\n")
    return sty_gen_inference_module_results

###############################################################################################################
## ---------------------     STORY GENERATOR ENDS                  STORY GENERATOR ENDS   ---------------------
###############################################################################################################

In [22]:
## Parameters to generate the story
## NOTE: nsamples % batch_size == 0
SG_LENGTH = 300
SG_TEMPERATURE = 0.95
SG_NSAMPLES = 1
SG_BATCH_SIZE = 1
RUN_NAME = r"Run2_File11_2_checkpoint_run2"
RETURN_LIST = False
DEST_PATH = None
DEBUG_SWITCH = False

## seed value - story 6
seed_for_styGen = r'Woman is sitting on couch with her cell phone.'

## check samples and batch size values are compatible
assert (SG_NSAMPLES % SG_BATCH_SIZE == 0) , f"Values for NSAMPLES and BATCH_SIZE incompatible. " \
        f"NSAMPLES={SG_NSAMPLES} % SG_BATCH_SIZE={SG_BATCH_SIZE} != 0"

SG_PARAMS = (seed_for_styGen, SG_LENGTH, SG_TEMPERATURE, SG_NSAMPLES, SG_BATCH_SIZE, RUN_NAME, RETURN_LIST, DEST_PATH)

story_text = sty_gen_inference_functionality(SG_PARAMS, DEBUG_SWITCH)

print(f"\ntype = {type(story_text)}\n\nOutput =\n{story_text}")


TF session started


checkpoint_path = /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2


Loading checkpoint :: /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000

INFO:tensorflow:Restoring parameters from /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000


Generating the story using 'prefix' = Woman is sitting on couch with her cell phone.

Woman is sitting on couch with her cell phone.
She can be annoying -- she and her fashion.
I wonder if she will have any idea -- no posture, or attitude whatever.
Any way she is, her weight is such a troublesome thing to me.
Every fine and dainty thing she asks me to lay down among it -- feet, arms, or books that I did not mind because I thought I would future for a horse, if I knew how to ride.
She is a good advocate.
Nothing frustrates her like my blemishes I have worn down my beard -- it was so smooth and soft, and she was so kind to me w

In [25]:
## Parameters to generate the story
## NOTE: nsamples % batch_size == 0
SG_LENGTH = 300
SG_TEMPERATURE = 0.95
SG_NSAMPLES = 1
SG_BATCH_SIZE = 1
RUN_NAME = r"Run2_File11_2_checkpoint_run2"
RETURN_LIST = False
DEST_PATH = None
DEBUG_SWITCH = False

## seed value - story 6
seed_for_styGen = r'Young boy standing in front of tv playing video game.'

## check samples and batch size values are compatible
assert (SG_NSAMPLES % SG_BATCH_SIZE == 0) , f"Values for NSAMPLES and BATCH_SIZE incompatible. " \
        f"NSAMPLES={SG_NSAMPLES} % SG_BATCH_SIZE={SG_BATCH_SIZE} != 0"

SG_PARAMS = (seed_for_styGen, SG_LENGTH, SG_TEMPERATURE, SG_NSAMPLES, SG_BATCH_SIZE, RUN_NAME, RETURN_LIST, DEST_PATH)

story_text = sty_gen_inference_functionality(SG_PARAMS, DEBUG_SWITCH)

print(f"\ntype = {type(story_text)}\n\nOutput =\n{story_text}")


TF session started


checkpoint_path = /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2


Loading checkpoint :: /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000

INFO:tensorflow:Restoring parameters from /home/rohit/PyWDUbuntu/thesis/StyGen/checkpoint/Run2_File11_2_checkpoint_run2/model-17000


Generating the story using 'prefix' = Young boy standing in front of tv playing video game.

Young boy standing in front of tv playing video game.
Sammy broke his way down up to his room and devoured everything in to take the dare.
Then he began to sing with a beaming voice.
`` Let the boy go and play with Reddy Fox ! ''
`` What da -- forget it, Sammy Brown ! ''
cried Reddy Fox.
`` Sammy Brown is a man that can not be leavin ' the Green Meadows alone . ''
That was not to Sammy Brown, though he knew it.
They were all afraid, even the ones to his face like Sammy.
Sammy had been looking out for Reddy Fox for ages.
It was never b